# Simple Decision Tree Classifier

In [3]:
import pandas as pd
import numpy as np
import os
import csv
from operator import itemgetter

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier

import nDCG

## Load dataset and choose features

In [4]:
# load dataset
df = pd.read_pickle('cleaned_dataset')

In [5]:
# add column with relevance scores
df['relevance'] = df['booking_bool'] + df['click_bool']

# change relevance column with relevance score (1 if clicked, 5 if booked)
df['relevance'] = df['relevance'].map({0:0, 1:1, 2:5})

# select features to train model with
selected_features_a = ['value_for_money','price_rank', 'star_diff_from_historic_mean','comp1_rate_percent_diff',\
                     'srch_query_affinity_score','srch_room_count','price_diff_from_historic_mean', \
                     'visitor_hist_adr_usd', 'site_id', 'prop_review_score', 'prop_location_score1', \
                     'prop_location_score2', 'price_usd', 'visitor_hist_starrating','srch_length_of_stay',\
                     'srch_adults_count', 'srch_children_count', 'srch_saturday_night_bool', 'srch_booking_window',\
                     'orig_destination_distance', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate',\
                     'comp2_inv','comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
                     'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
                     'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
                     'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
                     'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
                     'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
                     'comp8_rate_percent_diff','prop_log_historical_price', 'prop_brand_bool', 'promotion_flag']
selected_features = ['site_id', 'visitor_location_country_id',
                       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
                       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
                       'prop_location_score1', 'prop_location_score2',
                       'prop_log_historical_price', 'price_usd', 'promotion_flag',
                       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
                       'srch_adults_count', 'srch_children_count', 'srch_room_count',
                       'srch_saturday_night_bool', 'srch_query_affinity_score',
                       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
                       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
                       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
                       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
                       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
                       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
                       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
                       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
                       'comp8_rate_percent_diff',  'price_diff_from_historic_mean',
                       'star_diff_from_historic_mean', 'price_rank', 'star_rank',
                       'value_for_money', 'price_usd_normalized']



features_to_keep = ['relevance', 'srch_id', 'prop_id', 'booking_bool']

all_possible_features = selected_features + features_to_keep
df_select = df.loc[:,all_possible_features]

In [6]:
df_select=df_select.dropna()
df_select

,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,price_diff_from_historic_mean,star_diff_from_historic_mean,price_rank,star_rank,value_for_money,price_usd_normalized,relevance,srch_id,prop_id,booking_bool
0,12,187,3.46,154.27,219,3,3.5,1,2.83,0.0438,...,-25.28,-150.94,3.0,3.0,0.033407,0.037624,0,1,893,0
1,12,187,3.46,154.27,219,4,4.0,1,2.20,0.0149,...,-25.28,-150.94,14.0,14.0,0.023427,0.165564,0,1,10404,0
2,12,187,3.46,154.27,219,3,4.5,1,2.20,0.0245,...,-25.28,-150.94,15.0,15.0,0.025028,0.183135,0,1,21315,0
3,12,187,3.46,154.27,219,2,4.0,1,2.83,0.0125,...,-25.28,-150.94,22.0,22.0,0.006656,1.000000,0,1,27348,0
4,12,187,3.46,154.27,219,4,3.5,1,2.64,0.1241,...,-25.28,-150.94,11.0,11.0,0.024377,0.112891,0,1,29604,0
5,12,187,3.46,154.27,219,4,4.5,1,2.77,0.1302,...,-25.28,-150.94,19.0,19.0,0.023039,0.213234,0,1,30184,0
6,12,187,3.46,154.27,219,3,3.5,1,2.20,0.0356,...,-25.28,-150.94,8.0,8.0,0.027058,0.085294,0,1,44147,0
7,12,187,3.46,154.27,219,2,0.0,0,1.61,0.0692,...,-25.28,-150.94,1.0,1.0,0.000000,0.000000,0,1,50984,0
8,12,187,3.46,154.27,219,4,4.0,1,2.56,0.1238,...,-25.28,-150.94,12.0,12.0,0.026658,0.125439,0,1,53341,0
9,12,187,3.46,154.27,219,4,4.0,1,2.83,0.1028,...,-25.28,-150.94,21.0,21.0,0.014251,0.378799,0,1,56880,0


## Create training and test set taking into account search IDs

In [7]:
# create list of ids in random order
id_list = df_select.srch_id.unique()
np.random.shuffle(id_list)

# for reproducibility 
# np.random.seed(1337)

# training - test fraction is 80/20
id_train = int(len(id_list) * 0.8)
training_id = id_list[:id_train]
test_id = id_list[id_train:]

# divide dataset in test and training set
train_data = df_select.loc[df.srch_id.isin(training_id)]
test_data = df_select.loc[df.srch_id.isin(test_id)]

## Training the model

In [6]:
# select features and target variable
features = train_data[selected_features].values
target = train_data["booking_bool"].values

# train randomforest classifier on training data
classifier = RandomForestClassifier(n_estimators=50, 
                                    verbose=2,
                                    n_jobs=1,
                                    min_samples_split=10,
                                    random_state=1)
classifier.fit(features, target)

building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  3.5min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=1, verbose=2, warm_start=False)

## Predict test set

In [9]:
def write_submission(recommendations, submission_file):
    """
    Function which writes submission, ordered on the probability obtained by the model.
    The columns are SearchId, PropertyId and Relevance    
    """
    global rows
    submission_path = "/home/marleen/projects/DM2018/Assignment2/Bench_Results/"+submission_file
    rows = [(srch_id, prop_id, relevance)
        for srch_id, prop_id, relevance, rank_float
        in sorted(recommendations, key=itemgetter(0,3))]
    writer = csv.writer(open(submission_path, "w"), lineterminator="\n")
    writer.writerow(("SearchId", "PropertyId", "Relevance"))
    writer.writerows(rows)
    

In [8]:
# test model with features of test dataset
features = test_data[selected_features].values

# make predictions with trained model
predictions = classifier.predict_proba(features)[:,1]
predictions = list(-1.0*predictions)
recommendations = list(zip(test_data["srch_id"], test_data["prop_id"], test_data['relevance'], predictions))

# write predictions to csv file
write_submission(recommendations, "results.csv")

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.8s finished


In [14]:
# compute nDCG for results of test set
path_results = "Bench_Results/results.csv"
print(nDCG.compute_ndcg(path_results))

0.447783211234


In [13]:
results = pd.read_csv(path_results)
import math

print(results.columns)

pred_scores = [i[2] for i in rows]
# eval_df = train_df[['srch_id', 'booking_bool', 'click_bool']]
results['score'] = pred_scores

logger = lambda x: math.log(x + 1, 2)
results['log_rank'] = results['score'].map(logger)

# eval_df['log_rank'] = eval_df.groupby(by = 'srch_id')['score'].rank(ascending = False).map(logger)

book_dcg = (eval_df['booking_bool'] * 31.0 / eval_df['log_rank']).sum() #where 2 ** 5 - 1.0 = 31.0
book_idcg = (31.0 * eval_df['booking_bool']).sum()

click_dcg = (eval_df['click_bool'] * (eval_df['booking_bool'] == 0) / eval_df['log_rank']).sum()

# # Max number of clicks in training set is 30.
# # Calculate the 31 different contributions to IDCG that 0 to 30 clicks have
# # and put in dict {num of click: IDCG value}.
# disc = [1.0 / math.log(i + 1, 2) if i != 0 else 0 for i in range(31)]
# disc_dict = { i: np.array(disc).cumsum()[i] for i in range(31)}

# # Map the number of clicks to its IDCG and subtract off any clicks due to bookings
# # since these were accounted for in book_idcg.
# click_idcg = (eval_df.groupby(by = 'srch_id')['click_bool'].sum().map(disc_dict) - eval_df.groupby(by = 'srch_id')['booking_bool'].sum()).sum()

# return (book_dcg + click_dcg) / (book_idcg + click_idcg)



Index(['SearchId', 'PropertyId', 'Relevance'], dtype='object')


ValueError: Length of values does not match length of index

## Compare with random results

In [ ]:
# write file with random results
# np.random.seed(1)
ordinals = np.arange(len(test_data))
np.random.shuffle(ordinals)
recommendations = zip(test_data["srch_id"], test_data["prop_id"], test_data['relevance'], ordinals)
write_submission(recommendations, "randomBenchmark.csv")

In [12]:
# compute nDCG 
path_results = "Bench_Results/randomBenchmark.csv"
print(nDCG.compute_ndcg(path_results))

0.348456445719


## Compare with perfect results

In [10]:
# write file with perfect results
ordinals = np.arange(len(test_data))
recommendations = zip(test_data["srch_id"], test_data["prop_id"], test_data['relevance'], ordinals)
write_submission(recommendations, "testOrderBenchmark.csv")

In [11]:
# compute nDCG 
path_results = "Bench_Results/testOrderBenchmark.csv"
print(nDCG.compute_ndcg(path_results))

0.349424231581
